In [1]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2


In [ ]:

import numpy as np
print(np.__version__)
from mirage import WordCountingChunkingAlgorithm, FolderRawStorage, WhooshChunkStorage
from mirage import HuggingFaceEmbedder, TextNormalizer
from mirage import NatashaSentenceChunking, SemanticChunking

1.26.4


c:\Users\Bulkin\MIRAGE\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embedder = HuggingFaceEmbedder(model_name='BAAI/bge-m3', normalizer=True)

In [4]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data_test')
 # тут заменить на нужную папку
chunks = WhooshChunkStorage(scoring_function="BM25F", normalizer=True)
# algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)


In [5]:

algorithm = SemanticChunking(raw_storage=documents, chunk_storage=chunks, embedder=embedder, threshold=0.25, max_chunk_size=1000)
algorithm.execute(visualize=True)

Splitting text into the sentences


611it [00:00, 39683.48it/s]


creating embeddings of the sentences for semantic grouping


100%|██████████| 611/611 [01:23<00:00,  7.30it/s]


Adding chunks to the storage


100%|██████████| 50/50 [00:01<00:00, 41.63it/s]


1

In [6]:
import random
index = chunks.get_indexes()[random.randint(1, 50)]
print(len(chunks[index]))
chunks[index]

889


'— Для меня большая честь работать вместе с вами, мистер Холме, — с искренним чувством ответил инспектор. — В таком случае, я хотел бы, не откладывая, выслушать свидетелей и осмотреть то место, где было совершено преступление. Инспектор Мартин был настолько умен, что позволил моему приятелю поступать по-своему. Сам он ограничился тем, что внимательно следил за его работой. Местный врач, седобородый старик, только что вышел из комнаты миссис Хилтон Кьюбитт и сообщил, что ее положение серьезно, но не безнадежно; однако в сознание она придет, вероятно, не скоро, так как пуля задела мозг. На вопрос, сама ли она в себя выстрелила или в нее выстрелил кто-нибудь другой, он не решился дать определенный ответ. Во всяком случае, выстрел был сделан с очень близкого расстояния. В комнате нашли всего один револьвер; оба ствола были пусты. Мистер Хилтон Кьюбитт убит выстрелом прямо в сердце.'

In [7]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex
from mirage import FaissIndexFlatL2, FaissIndexIVFPQR
vector_index = L2RAMVectorIndex(dimensionality=embedder.get_dimensionality())
embedder.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 50/50 [00:27<00:00,  1.84it/s]


In [26]:
query ='кто убил прекрасную молодую сипатичную женщину?'
query_vector = embedder.embed(text=query)
# print(query_vector)
vector_answer = vector_index.query(
    query_vector, top_k=5
)
print(vector_answer)


[QueryResult(score=0.4861423075199127, chunk_storage_key='b18d1d05-ec8e-449c-821d-47da92d64c47', vector=array([-0.0075999 ,  0.0326061 ,  0.00334911, ...,  0.00832559,
       -0.01666468,  0.00035527], dtype=float32)), QueryResult(score=0.4567331075668335, chunk_storage_key='c9d74173-066f-46a2-8990-f2f1049c2a6d', vector=array([ 0.00618527,  0.03533252, -0.02911684, ...,  0.02144764,
       -0.04102236,  0.00326473], dtype=float32)), QueryResult(score=0.45087435841560364, chunk_storage_key='ae0a58c7-62eb-4f2a-a845-d2041263e706', vector=array([-0.02809869,  0.00841984, -0.02139766, ...,  0.0196938 ,
        0.00812328,  0.00494469], dtype=float32)), QueryResult(score=0.4437407851219177, chunk_storage_key='d1297073-6708-45eb-aa53-60103a9345bb', vector=array([-0.01034235,  0.02876878, -0.03985514, ...,  0.06146912,
       -0.0017426 ,  0.01379395], dtype=float32)), QueryResult(score=0.4376612603664398, chunk_storage_key='5e8bbb74-7de5-437f-9f1d-471666dc38e0', vector=array([-0.0154899 ,  0.

In [46]:
chunks.get_texts_for_search_results(vector_answer)

['— Это дело обещает много любопытного и необычайного, — сказал он. — Вы уже кое-что рассказали мне в своем письме, мистер Хилтон Кьюбитт, но я был бы очень вам признателен, если бы вы любезно согласились повторить свой рассказ, чтобы дать возможность послушать его моему другу, доктору Уотсону. — Я плохой рассказчик, — сказал наш гость, нервно сжимая и разжимая свои большие сильные руки. — Если в моем рассказе вам что-нибудь покажется неясным, задавайте мне, пожалуйста, вопросы. Начну с того, что в прошлом году я женился… Но предварительно я должен сказать, что хотя я человек небогатый, наш род живет в Ридлинг-Торпе уже в течение пяти столетий и считается самым знатным родом во всем Норфолкском графстве. В прошлом году я приехал в Лондон на праздники и остановился в меблированных комнатах на Рэссел-сквере, потому что там остановился Паркер, священник нашего прихода. В этих меблированных комнатах жила молодая американская леди, по фамилии Патрик, Илей Патрик. Мы с ней скоро подружились.

In [38]:
from mirage.inference.quorums.RusVectoresQuorum import RusVectoresQuorum


quorum = RusVectoresQuorum(
    chunk_storage=chunks, 
    path_to_model='mirage\inference\quorums\model.bin',
    POS_thresholds={"ADJ": 0.25, "VERB": 0.23},
    visualize=False,
    max_combinations=100,
    max_synonims=5,
    max_entries=100)
fulltext_search_answer = quorum.query(query)
fulltext_search_answer

[QueryResult(score=8.584362381613307, chunk_storage_key='e0396c6a-9203-4053-826f-7bbff7132c00', vector=None),
 QueryResult(score=4.479393901730816, chunk_storage_key='c9d74173-066f-46a2-8990-f2f1049c2a6d', vector=None),
 QueryResult(score=4.12269114898283, chunk_storage_key='773f79d2-14d6-401e-8552-3ce2268ed28b', vector=None),
 QueryResult(score=3.9832399185359524, chunk_storage_key='b18d1d05-ec8e-449c-821d-47da92d64c47', vector=None),
 QueryResult(score=3.853428185762432, chunk_storage_key='9d13f6dd-ba55-48a6-b710-67cb439c793e', vector=None),
 QueryResult(score=3.7310737033203254, chunk_storage_key='d7b3ac31-d47e-46be-aa3a-7b5025c7920b', vector=None),
 QueryResult(score=3.28772957412409, chunk_storage_key='71e5eca5-3a78-4832-af7f-4b45a3869910', vector=None),
 QueryResult(score=2.998487811394627, chunk_storage_key='f92db50f-ea37-48c8-a5ed-96881c6773a2', vector=None),
 QueryResult(score=2.998487811394627, chunk_storage_key='f17b4f65-7ede-46db-8287-38f9f8c0b8a9', vector=None),
 QueryResu

In [39]:
chunks.get_texts_for_search_results(fulltext_search_answer)

['— Он ранен, а не она! Разве у кого-нибудь хватило бы духу ранить маленькую Илей? Я угрожал ей, да простит меня бог, но я не коснулся бы ни одного волоса на ее прекрасной голове. Возьмите свои слова обратно — эй, вы! Скажите, что она не ранена! — Она была найдена тяжело раненной возле своего мертвого мужа. С глубоким стоном он опустился на диван и закрыл лицо руками. Он молчал целых пять минут. Затем открыл лицо и заговорил с холодным спокойствием отчаяния. — Мне нечего скрывать от вас, джентльмены, — сказал он. — Я стрелял в него, но и он стрелял в меня, — следовательно, это нельзя назвать убийством. Если же вы думаете, что я в состоянии ранить ту женщину, значит, вы не знаете ни ее, ни меня. Ни один мужчина никогда не любил ни одной женщины так, как я любил ее. Я имел все права на нее. Она была мне предназначена уже много лет назад. На каком основании этот англичанин встал между нами? Я первый получил на нее права, и я требовал только того, что мне принадлежит.',
 'Не прошло и месяц

In [40]:
from mirage.inference.rerankers.L2Reranker import L2Reranker
from mirage.inference.rerankers.WeightedProductReranker import WeightedProductReranker
from mirage.inference.rerankers.ReciprocalRankFusionReranker import ReciprocalRankFusionReranker
from mirage.inference.rerankers.BordaScoreReranker import BordaScoreReranker

rank_fusion = BordaScoreReranker()
ranked_answer = rank_fusion(fulltext_search_answer, vector_answer)
print(*ranked_answer, sep='\n')

score=21.0 chunk_storage_key='c9d74173-066f-46a2-8990-f2f1049c2a6d' vector=None
score=20.0 chunk_storage_key='b18d1d05-ec8e-449c-821d-47da92d64c47' vector=None
score=18.0 chunk_storage_key='e0396c6a-9203-4053-826f-7bbff7132c00' vector=None
score=15.0 chunk_storage_key='773f79d2-14d6-401e-8552-3ce2268ed28b' vector=None
score=12.0 chunk_storage_key='9d13f6dd-ba55-48a6-b710-67cb439c793e' vector=None
score=11.0 chunk_storage_key='ae0a58c7-62eb-4f2a-a845-d2041263e706' vector=None
score=10.0 chunk_storage_key='d7b3ac31-d47e-46be-aa3a-7b5025c7920b' vector=None
score=10.0 chunk_storage_key='d1297073-6708-45eb-aa53-60103a9345bb' vector=None
score=9.0 chunk_storage_key='5e8bbb74-7de5-437f-9f1d-471666dc38e0' vector=None
score=8.0 chunk_storage_key='71e5eca5-3a78-4832-af7f-4b45a3869910' vector=None
score=6.0 chunk_storage_key='f92db50f-ea37-48c8-a5ed-96881c6773a2' vector=None
score=4.0 chunk_storage_key='f17b4f65-7ede-46db-8287-38f9f8c0b8a9' vector=None
score=2.0 chunk_storage_key='7e7e351a-66ff-4

In [41]:
chunks.get_texts_for_search_results(ranked_answer)

['Не прошло и месяца, как я полюбил ее самой пылкой любовью. Мы тихонько повенчались и уехали ко мне в Норфолк. Вам, вероятно, кажется странным, мистер Холмс, что человек хорошего старинного рода вступает в брак с женщиной, ничего не зная о ее прошлом и о ее семье. Но если бы вы увидели ее и узнали, вам нетрудно было бы меня понять. Она была очень прямодушна со мной, моя Илей, она предоставляла мне полную возможность отказаться от свадьбы, если я захочу. «У меня в моей прежней жизни были очень неприятные знакомства, — говорила она, — я хочу позабыть о них. Я не желаю возвращаться к своему прошлому, потому что всякое воспоминание причиняет мне боль. Если ты на мне женишься, Хилтон, ты женишься на женщине, которая сама ничего постыдного не совершила, но ты должен поверить мне на слово и позволить умолчать обо всем, что было со мною до того, как я стала твоей.',
 '— Это дело обещает много любопытного и необычайного, — сказал он. — Вы уже кое-что рассказали мне в своем письме, мистер Хилто

In [13]:
from mirage.inference.prompters.APILLM import LLM
from mirage.inference.prompters.LLMGroq import GroqLLM
llm = GroqLLM()
llm.do_request(query="What this text about?",#query,
               chunk_storage=chunks,
            #    indexes=[element.chunk_storage_key for element in ranked_answer],
               indexes=chunks.get_indexes()[9:12],
               prompt_template=None)

ModuleNotFoundError: No module named 'groq'